In [2]:
import pandas as pd
from dask.array.image import imread
import napari

processed_data_location = "E:/Science/Data/Lund Zenodo/Processed/"
glob_name

In [26]:
example_file_name = processed_data_location + 'label_image_lund_i000027_oi_000165.tif'
index = example_file_name[56:62]
int(index)

'000027'

In [27]:
example_file_name = processed_data_location + 'rescaled_lund_i000184_oi_000824.tif'
index = example_file_name[53:59]
int(index)

184

In [28]:
def custom_imread(filename):
    from skimage.io import imread
    
    if 'label' in filename:
        ind = int(filename[56:62])
    elif 'rescaled' in filename:
        ind = int(filename[53:59])
    else:
        print('no files with the right format')
    load_image = metadata['load_image'].tolist()[ind]
    if load_image:
        return imread(filename)


In [29]:
metadata = pd.read_csv(processed_data_location + 'advanced_metadata.csv')
metadata

,Unnamed: 0.1,Unnamed: 0,time_in_s,original_index,index,segmented_cells,duplicate_oi,load_image
0,0,0,0.0000,0,0,111865.0,False,False
1,1,1,419.9778,5,1,112364.0,False,False
2,2,2,779.9945,7,2,108871.0,False,False
3,3,3,960.0141,8,3,114159.0,False,False
4,4,4,1320.0067,10,4,107069.0,False,False
...,...,...,...,...,...,...,...,...
427,427,427,128139.6458,2388,427,1581.0,True,False
428,428,428,128739.9586,2389,428,1467.0,False,True
429,429,429,128739.9586,2389,429,1467.0,True,False
430,430,430,129339.7183,2390,430,1493.0,False,True


In [30]:
from dask.array.image import imread
labels_dask = imread(processed_data_location + 'label_image_lund_i*.tif', imread=custom_imread)

AttributeError: 'NoneType' object has no attribute 'shape'

In [35]:
def custom_dask_loader(filename_glob,metadata):
    from skimage.io import imread
    from skimage.io.collection import alphanumeric_key
    from dask import delayed
    import dask.array as da
    from glob import glob

    filenames = sorted(glob(filename_glob), key=alphanumeric_key)
    if 'label' in filename_glob:
        filenames_filtered = [name for name in filenames if metadata['load_image'].tolist()[int(name[56:62])]]
    elif 'rescaled' in filename_glob:
        filenames_filtered = [name for name in filenames if metadata['load_image'].tolist()[int(name[53:59])]]
    else:
        print('no files with the right format')
        
    sample = imread(filenames[0])
    
    lazy_imread = delayed(imread)  # lazy reader
    lazy_arrays = [lazy_imread(fn) for fn in filenames_filtered]
    dask_arrays = [
        da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
        for delayed_reader in lazy_arrays
    ]
    # Stack into one large dask.array
    stack = da.stack(dask_arrays, axis=0)
    
    return stack

labels_stack = custom_dask_loader(processed_data_location + 'label_image_lund_i*.tif', metadata)
labels_stack

,Array,Chunk
Bytes,68.20 GiB,204.80 MiB
Shape,"(341, 213, 710, 355)","(1, 213, 710, 355)"
Count,1023 Tasks,341 Chunks
Type,uint32,numpy.ndarray


In [36]:
rescaled_stack =custom_dask_loader(processed_data_location + 'rescaled_lund_i*.tif', metadata)
rescaled_stack

,Array,Chunk
Bytes,68.20 GiB,204.80 MiB
Shape,"(341, 213, 710, 355)","(1, 213, 710, 355)"
Count,1023 Tasks,341 Chunks
Type,float32,numpy.ndarray


In [38]:
rescaled_crop = rescaled_stack[140:210]
rescaled_crop

,Array,Chunk
Bytes,14.00 GiB,204.80 MiB
Shape,"(70, 213, 710, 355)","(1, 213, 710, 355)"
Count,1093 Tasks,70 Chunks
Type,float32,numpy.ndarray


In [39]:
labels_crop = labels_stack[140:210]
labels_crop

,Array,Chunk
Bytes,14.00 GiB,204.80 MiB
Shape,"(70, 213, 710, 355)","(1, 213, 710, 355)"
Count,1093 Tasks,70 Chunks
Type,uint32,numpy.ndarray


In [40]:
viewer = napari.Viewer()
viewer.add_image(rescaled_crop, contrast_limits=[0,2000], multiscale=False)
viewer.add_labels(labels_crop)

C:\Users\ryans\anaconda3\envs\ncp_dev_v2\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'labels_crop' at 0x24b2d658b50>